In [15]:
from tensorflow import keras

import time
import sys
from pathlib import Path
import numpy as np
import os

# Absolute path is needed to load libraries 
ROOT_PATH = os.path.abspath('')
sys.path.append(ROOT_PATH + '/lib')

# from lib.Kmeans_lib import *
# from lib.EvalMetrics import *

from lib.simulation_lib import save_plots
from lib.CustomLayer_lib import Custom_Layer
from lib.utils import create_dataset, RunOneEpoch, TrainSettings

In [16]:
# Define settings

settings_train = TrainSettings()
settings_train.verbosity = 'NONE'
settings_train.fill_cmtx = False
settings_train.save_output = False
settings_train.save_plots = False
settings_train.mode = 'Train'

settings_test = TrainSettings()
settings_test.verbosity = 'EOEINFO'
settings_test.fill_cmtx = True
settings_test.save_output = True
settings_test.save_plots = True
settings_test.mode = 'Test'

In [17]:
import shutil
shutil.rmtree('Results/', ignore_errors = True)

In [19]:
# File structure:
# /Results/dataset_

if True:
    list_datasets = [[3500, 500]]# Format (n_train, n_test)
    list_features = [50] # [10, 30, 50, 100]
    list_batches = [50] #[20, 40, 100]# [5, 10, 20, 40, 100]
    iterations = 3

for n_train, n_test in list_datasets:
    for idx in range(1, iterations+1):

        print("############# STARTING DATASET {} ############".format(idx))

        # Create directory and log file
        if settings_test.save_output == True:      
            RESULTS_PATH = 'Results/ds{}_{}_{}/'.format(idx,n_train, n_test)
            os.makedirs(RESULTS_PATH, exist_ok = True)
            settings_test.save_path = RESULTS_PATH
            
            with open(RESULTS_PATH + 'output_log.txt', 'w') as new_file:  # overwrite if already exists
                # File headers
                new_file.write("# Dataset: {}, n_train = {}, n_test = {} \n".format(idx,n_train, n_test))
                new_file.write("# \n")
                new_file.write("# FEATURES\t; BATCH_SZ\t; CST_ERRS\t; CST_ACC\t; MDL_ERRS\t; MDL_ACC\t;\t TIME\t; CST_ERR_ARRAY\t; MDL_ERR_ARRAY\n")
                new_file.close()

        # Create dataset
        digits_train, label_digits_train, digits_test, label_digits_test = create_dataset(n_train, n_test)

        for n_feat in list_features:

            print("Starting with n_features: ", n_feat)

            #  Set model and features
            MODEL_PATH = 'Models/{}/'.format(n_feat)
            features_saved = np.loadtxt(MODEL_PATH + 'll_features.txt')
            labels_features_saved = np.loadtxt(MODEL_PATH + 'll_labels_features.txt').astype(int)
            keras_model = keras.models.load_model(MODEL_PATH + 'original_mnist_cnn.h5')  # Original model

            for batch_size in list_batches:
                
                print("Starting batch size: ", batch_size)

                # Initialize model
                Model = Custom_Layer(keras_model)
                Model.title = 'OL'
                Model.filename = 'OL'
                Model.l_rate = 0.01
                Model.update_batch_size = 10
                Model.ll_method = 'OL'
                Model.clustering_batch_size = batch_size

                startTime = time.time()
                
                # Train the model
                Model.settings = settings_train
                RunOneEpoch(Model, digits_train, label_digits_train,features_saved, labels_features_saved)

                # Test the model
                Model.settings = settings_test
                RunOneEpoch(Model, digits_test, label_digits_test,features_saved, labels_features_saved)
                
                endTime = time.time()

                # Save log
                # Unpack datalog output
                clust_err = Model.settings.datalog[0]
                model_err = Model.settings.datalog[1]
                clust_err_array = Model.settings.datalog[2]
                model_err_array = Model.settings.datalog[3]

                if Model.settings.save_output == True:
                    with open(RESULTS_PATH + 'output_log.txt', 'a') as new_file:  
                        new_file.write("\t{:3d} \t;\t {:3d} \t;\t".format(n_feat,batch_size))
                        new_file.write("{:3d} \t;\t {:.1%} \t;\t{:3d} \t;\t {:.1%} \t;\t".format(clust_err, 1-clust_err/n_test, model_err, 1-model_err/n_test))
                        new_file.write("{:.3f}\t;\t".format(endTime-startTime))
                        new_file.write("{}\t;\t{}\n".format(clust_err_array, model_err_array))
                    new_file.close()

                #  Save plots
                if Model.settings.save_plots == True:
                    save_plots(Model, RESULTS_PATH + '/Plots', 'Features_{}_Batch_{}'.format(n_feat, batch_size))


############# STARTING DATASET 1 ############
Starting with n_features:  50
Starting batch size:  50
Starting Train batch: 1/70
New digit detected -> 8
New digit detected -> 7
New digit detected -> 9
New digit detected -> 6
Starting Train batch: 2/70
Starting Train batch: 3/70
Starting Train batch: 4/70
Starting Train batch: 5/70
Starting Train batch: 6/70
Starting Train batch: 7/70
Starting Train batch: 8/70
Starting Train batch: 9/70
Starting Train batch: 10/70
Starting Train batch: 11/70
Starting Train batch: 12/70
Starting Train batch: 13/70
Starting Train batch: 14/70
Starting Train batch: 15/70
Starting Train batch: 16/70
Starting Train batch: 17/70
Starting Train batch: 18/70
Starting Train batch: 19/70
Starting Train batch: 20/70
Starting Train batch: 21/70
Starting Train batch: 22/70
Starting Train batch: 23/70
Starting Train batch: 24/70
Starting Train batch: 25/70
Starting Train batch: 26/70
Starting Train batch: 27/70
Starting Train batch: 28/70
Starting Train batch: 29/70
